In [36]:
import requests
import json
import mysql.connector


ModuleNotFoundError: No module named 'mysql'

In [29]:
url = 'https://www.pse.pl/transmissionMapService'
resp = requests.get(url=url).json()
print(resp)

{'status': '0', 'timestamp': 1711631641312, 'data': {'przesyly': [{'wartosc': -614, 'rownolegly': False, 'wartosc_plan': -600, 'id': 'SE'}, {'wartosc': 1490, 'rownolegly': True, 'wartosc_plan': 1405, 'id': 'DE'}, {'wartosc': -1174, 'rownolegly': True, 'wartosc_plan': -263, 'id': 'CZ'}, {'wartosc': -256, 'rownolegly': True, 'wartosc_plan': -542, 'id': 'SK'}, {'wartosc': 22, 'rownolegly': False, 'wartosc_plan': 0, 'id': 'UA'}, {'wartosc': 0, 'rownolegly': False, 'wartosc_plan': 0, 'id': 'LT'}], 'podsumowanie': {'wodne': 302, 'wiatrowe': 5845, 'PV': 4736, 'generacja': 23603, 'zapotrzebowanie': 23104, 'czestotliwosc': 50.023, 'inne': 0, 'cieplne': 12720}}}


In [34]:
respDump = json.dumps(resp)
data = json.loads(respDump)
print(data)
print(data['timestamp'])

{'status': '0', 'timestamp': 1711631641312, 'data': {'przesyly': [{'wartosc': -614, 'rownolegly': False, 'wartosc_plan': -600, 'id': 'SE'}, {'wartosc': 1490, 'rownolegly': True, 'wartosc_plan': 1405, 'id': 'DE'}, {'wartosc': -1174, 'rownolegly': True, 'wartosc_plan': -263, 'id': 'CZ'}, {'wartosc': -256, 'rownolegly': True, 'wartosc_plan': -542, 'id': 'SK'}, {'wartosc': 22, 'rownolegly': False, 'wartosc_plan': 0, 'id': 'UA'}, {'wartosc': 0, 'rownolegly': False, 'wartosc_plan': 0, 'id': 'LT'}], 'podsumowanie': {'wodne': 302, 'wiatrowe': 5845, 'PV': 4736, 'generacja': 23603, 'zapotrzebowanie': 23104, 'czestotliwosc': 50.023, 'inne': 0, 'cieplne': 12720}}}
1711631641312
